Import libraries

In [40]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

Get Wikipedia article and make beautiful soup

In [41]:
html = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(html, "html5lib")

Read soup into a dataframe and get postal code, borough and neighborhood

In [42]:
df = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhood'])
for row in soup.find('tbody').find_all('td'):
    code = row.p.b.text
    if row.p.b.next_sibling.next_sibling.text != 'Not assigned':
        borough = row.p.b.next_sibling.next_sibling.contents[0].string
        neighbourhood = row.p.b.next_sibling.next_sibling.text[row.p.b.next_sibling.next_sibling.text.find('(')+1:-1].replace(' /',', ').replace('(',',').replace(')',"")
        df = df.append({'PostalCode': code, 'Borough': borough, 'Neighborhood': neighbourhood}, ignore_index=True)
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, ..."


Print the number of rows and columns

In [43]:
df.shape

(103, 3)